# Create a Virtual Machine in CSC cPouta

This notebook shows how to create a virtual machine in cPouta.

> For your interest, cPouta and ePouta are the IaaS cloud services at CSC, known as Finnish IT center for science. The cPouta cloud is the public cloud which is easily accessible via the internet. The ePouta cloud is a virtual private cloud designed to meet the security requirements of handling sensitive data. Both the cPouta and ePouta clouds run on the OpenStack cloud software. The Pouta cloud services are suitable for most kinds of computational workloads and any other supporting services these workloads might need. More information can be found from [CSC's documentations of the Pouta services](https://docs.csc.fi/cloud/pouta/pouta-what-is/).

## First glance at cPouta

We will try to create and manage a virtual machine from our local computer using OpenStack CLI. If you are interested, you can start familiarizing with cPouta GUI reading [this guide](https://docs.csc.fi/cloud/pouta/launch-vm-from-web-gui/).

## Prerequisites

### 0. You should already have a CSC account and can access the CSC project of the course
If not, please navigate to the course Moodle page and find the invitation link. Then follow the instructions to create an CSC account and apply for the access to the CSC project of the course. 

### 1. Configure your terminal environment for OpenStack
To be able to connect to the CSC system and cPouta, we need to get some credentials.
We can set them using a script provided in the OpenStack web interface.
You can download this script at the following URL after logging in. **Remember to switch to project_2008075 if you have multiple CSC projects**:

https://pouta.csc.fi/dashboard/project/api_access/

To download this in the web interface, navigate to the API Access section, and download both files.

![api_access](./images/csc-pouta-openstack-api_access.png)

If you already access the cPouta dashboard, these links might work to download the files directly:

> https://pouta.csc.fi/dashboard/project/api_access/openrc/
> https://pouta.csc.fi/dashboard/project/api_access/clouds.yaml/

Put the files in the current directory (setup/1_create_vm). 
> If you are using Firefox pre-installed in the Ubuntu VM, the browser should ask you where to put the downloaded files. If this is not the case, this option can be enabled with the following steps:
> 1. Open the setting menu by clicking on the three horizontal lines (hamburger icon) in the top-right corner of the Firefox window.
> 2. Click on "Settings" and then "General".
> 3. Navigate to the "Files and Applications" section and check the "Always ask you where to save files" checkbox. Now you can download the files again. 

For simplicity, rename the `project_2008075-openrc.sh` file as `openrc`.

The openrc file will be necessary to use the OpenStack CLI (command line interface). While the cloud.yaml file is necessary to use OpenStack with the Python API.


### 2. Install OpenStack CLI (command line interface)

There is a quite complete guide to install the CLI [here](https://docs.csc.fi/cloud/pouta/install-client/), so if you need extra help you can use that one.
The following is a faster step-by-step guide. 

**NOTE**: Unfortunately we can't run the OpenStack CLI in the python notebook because we will need the credentials from the openrc file, so you will need to open a terminal and follow along with the terminal on the side. This will be easy if you use VSCode. 

Once you have the openrc script from the web UI, open a terminal and ensure you are under the `setup/1_create_vm` directory and add the environment variables in the terminal using the following commands:

```bash
# suppose you're now under pre_materials directory, you can enter the 1_create_vm directory by 
cd setup/1_create_vm

# then add environment variables
source openrc
```

You will be asked to enter a password. **Please use the password of your CSC account (not your university account)**. After doing this, the current terminal session will have the proper environment variables for using the command line tools. Unfortunately, you will not get any error even if you enter a wrong password, but a wrong password will cause errors in later procedures. To avoid this, you can print the password you just entered by running the following command in the same terminal and check if the printed password is correct:
```bash
echo $OS_PASSWORD
``` 
**⚠️ NOTE:** You need to do this again if you open a new terminal during setting up a VM in cPouta.


Before running the code in the cells below, make sure you are using "mlops_eng" as the Jupyter notebook kernel. You can configure the kernel by clicking the "Select Kernel" option at the top right of the notebook:

<img src="./images/select-kernel.png" width=800>

Install the openstack CLI:

In [1]:
%pip install python-openstackclient~=6.2.0

Note: you may need to restart the kernel to use updated packages.


### 3. Install OpenStack's Python client SDK

In [2]:
%pip install openstacksdk~=1.2

Note: you may need to restart the kernel to use updated packages.


### 4. Configure the credentials that OpenStack's Python client needs for connecting to cPouta

Be sure to have downloaded the `clouds.yaml` file from the "prerequisites" chapter.

After that, you need to create a `secure.yaml` file in the current directory with your password in it.

**Disclaimer**: this solution is not secure, but OpenStack doesn't give other safer options. Be sure to delete this file after you are done and most importantly **NEVER COMMIT** the `secure.yaml`.

Below there is a script to create the file automatically. Run it and update the password with your CSC password in the `secure.yaml` file. The file is under the same directory as this notebook (i.e., setup/1_create_vm).

**Note**: Your CSC password is not the password of your university credentials, it's the one you created when you signed up to CSC after you had logged in using your university credentials for the first time. 

In [3]:
!echo -e "clouds:\n  openstack:\n    auth:\n      password: ******" > secure.yaml

Now update the password in [secure.yaml](./secure.yaml).

As long as the configuration files (clouds.yaml and secure.yaml) are in one of the following paths, the connection should work automatically:

- Current Directory
- ~/.config/openstack
- /etc/openstack

So we can start a connection to cPouta and create a virtual machine there.

## Create a virtual machine

We will mainly use the Python client in this notebook. 

For your interest, the complete guide on how to work with the CLI is [here](https://docs.csc.fi/cloud/pouta/command-line-tools/).

In [29]:
import openstack
from openstack.connection import Connection

# Initialize and turn off debug logging
openstack.enable_logging(debug=False)

# Initialize the connection to the cloud
conn: Connection = openstack.connect(cloud="openstack")


First of all, we need to create a key pair:

In [30]:
import os
from openstack.compute.v2.keypair import Keypair


def create_keypair(conn: Connection, keypair_name: str) -> Keypair:
    keypair = conn.compute.find_keypair(keypair_name)

    if not keypair:
        print("Creating Key Pair...")
        keypair = conn.compute.create_keypair(name=keypair_name)
        ssh_dir_name = os.path.join(os.environ["HOME"], ".ssh")
        
        # create .ssh folder under your home directory if not existing
        os.makedirs(ssh_dir_name, exist_ok=True)
        
        private_keypair_file_path = os.path.join(
            ssh_dir_name, keypair_name
        )
        
        with open(private_keypair_file_path, "w") as f:
            f.write("%s" % keypair.private_key)

        os.chmod(private_keypair_file_path, 0o400)
        print("Done.")
    else:
        print(f"The keypair {keypair_name} is already in the system.")

    return keypair


keypair = create_keypair(conn, f"{conn.auth['username']}-tutorial")
print(f"\n\tKey pair: {keypair.name} - {keypair.location.project.name}\n")


Creating Key Pair...
Done.

	Key pair: samusyrj-tutorial - project_2008075



You will see the keys created:

![keypair](./images/tutorial-openstack-keypair.png)

**Note**: If you want to delete an existing keypair and create a new one, remember to delete both the public key in cPouta and the private key in the `.ssh` folder under your home directory. 

Now we can try to create a virtual machine (sometimes referred to as a server in the following instructions) in cPouta. To do that, we need to define some details such as network, type of instance and image to start from.

Let's print the ones that are already available:

In [31]:
print("Available standard instances types:")
all_flavors = conn.compute.flavors()
for flavor in sorted(all_flavors, key=lambda x: x.name):
    if flavor.name.startswith("standard"):
        print(" - " + flavor.name)

print("\nAvailable networks:")
all_networks = conn.network.networks()
for network in all_networks:
    print(" - " + network.name)


print("\nAvailable images:")
all_images = conn.compute.images()
for image in all_images:
    print(" - " + image.name)


Available standard instances types:
 - standard.3xlarge
 - standard.large
 - standard.medium
 - standard.small
 - standard.tiny
 - standard.xlarge
 - standard.xxlarge

Available networks:
 - public
 - project_2008075

Available images:
 - Ubuntu-18.04
 - CentOS-8-Stream
 - Ubuntu-22.04
 - Ubuntu-20.04
 - CentOS-7-Cuda
 - CentOS-7
 - cirros


So now we can pick one of each and define a function to create the server. This step will take a while. 

In [32]:
# The openstack object has no return type, so the IDE can't find the right type with intellisense
# We import them only for autocomplete in the IDE and for extra detail, but they are not needed
from openstack.compute.v2.image import Image
from openstack.compute.v2.flavor import Flavor
from openstack.network.v2.network import Network
from openstack.compute.v2.server import Server


def create_server(
    conn: Connection,
    server_name: str,
    image_name: str,
    instance_flavor: str,
    network_name: str,
    keypair: Keypair,
) -> Server:
    server = conn.compute.find_server(server_name)
    if server is None:
        print("Creating Server:")
        image: Image = conn.image.find_image(image_name)
        flavor: Flavor = conn.compute.find_flavor(instance_flavor)
        network: Network = conn.network.find_network(network_name)
        server: Server = conn.compute.create_server(
            name=server_name,
            image_id=image.id,
            flavor_id=flavor.id,
            networks=[{"uuid": network.id}],
            key_name=keypair.name,
        )

        server: Server = conn.compute.wait_for_server(server)
        print(f"Server created: {server.name}")
    else:
        print(f"The server {server_name} is already running.")
    return server


server_name = f"{conn.auth['username']}-tutorial"
image_name = "Ubuntu-22.04"
instance_flavor = "standard.xlarge"
network_name = "project_2008075"

server = create_server(
    conn,
    server_name,
    image_name,
    instance_flavor,
    network_name,
    keypair,
)


Creating Server:
Server created: samusyrj-tutorial


The server should appear on the dashboard like below:

![server](./images/tutorial-openstack-server.png)

### Access the server through SSH

The IP that you see in the image is a private network IP.

Now that we have a server, we can give it a *Floating IP* (i.e. a public IP) so that we can access with SSH to run commands on it.

If we give a *Floating IP*, and we add the right security group and port forwarding rules to the server, we can access it from our local machine.

⚠️ **NOTE** ⚠️: the Python APIs are broken for this passage, so for now we need to run the commands on a terminal that has the right credentials.

Run the following python code and follow the instructions:

In [33]:
from openstack.network.v2.security_group import SecurityGroup
from openstack.network.v2.floating_ip import FloatingIP

server: Server = conn.compute.find_server(server_name, ignore_missing=False)
floating_ip: FloatingIP = conn.network.find_available_ip()
print(f"Available floating IP: {floating_ip.floating_ip_address}")
security_group: SecurityGroup = conn.network.find_security_group("ssh")
print(f"Security group found: {security_group.name}")

# NOTE: The APIs are broken, but it would be:
# conn.compute.add_floating_ip_to_server(server, floating_ip.floating_ip_address, floating_ip.fixed_ip_address)
# conn.compute.add_security_group_to_server(conn, security_group)
print("Run on your terminal these commands one at a time:")
print(f"\tSuppose you're in setup/1_create_vm")
print(f"\tsource openrc")
print(f"\topenstack server add floating ip {server.name} {floating_ip.floating_ip_address}")
print(f"\topenstack server add security group {server.name} {security_group.name}")
print(f"\topenstack server lock {server.name}") # prevent your VM from accidental deletion

Available floating IP: 128.214.254.171
Security group found: ssh
Run on your terminal these commands one at a time:
	Suppose you're in setup/1_create_vm
	source openrc
	openstack server add floating ip samusyrj-tutorial 128.214.254.171
	openstack server add security group samusyrj-tutorial ssh
	openstack server lock samusyrj-tutorial


In [34]:
# We assume your private key file is in $HOME/.ssh
import os
private_key_file_path = os.path.join(os.environ["HOME"], ".ssh", keypair.name)
username = "ubuntu" if "Ubuntu" in image_name else "root"
print("The server is now accessible with the following command:")
print(f"\n\tssh -i {private_key_file_path} {username}@{floating_ip.floating_ip_address}")

The server is now accessible with the following command:

	ssh -i /home/user/.ssh/samusyrj-tutorial ubuntu@128.214.254.171


NOTE: if you tried multiple times to run these cells and you already put the floating IP to your `known_hosts` file, you might get the following message when you try to ssh into the instance:

```text
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@    WARNING: REMOTE HOST IDENTIFICATION HAS CHANGED!     @
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
IT IS POSSIBLE THAT SOMEONE IS DOING SOMETHING NASTY!
Someone could be eavesdropping on you right now (man-in-the-middle attack)!
It is also possible that a host key has just been changed.
The fingerprint for the ED25519 key sent by the remote host is
SHA256:XXXXX
Please contact your system administrator.
Add correct host key in HOME/.ssh/known_hosts to get rid of this message.
Offending ECDSA key in HOME/.ssh/known_hosts:7
Host key for INSTANCE_IP has changed and you have requested strict checking.
Host key verification failed.
```

To solve this, you can simply get the `floating_ip` from the previous run, and run:

```bash
ssh-keygen -R <floating-ip>
```

and then try to ssh again into the instance.

After creating a VM in cPouta, save the notebook with the outputs (of running the cells) as you will need some of the outputs later. Keep the notebook open and continue with the "3. Install necessary tools and create a K8s cluster in the cPouta VM" section in the [main instructions](../../README.md).